<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>

# Laboratory work #2. Observation FHIR resource

A clinical coordinator  asked John about his complaints to clarify how we can help.
John explained, he doesn't feel well this week. It’s related to the temperature and high blood pressure.  This started to happen on Monday (10-20-2018).

## 1 Introduction

### 1.1 Import library

At the beginning we need to import libraries `fhirpy` and `os`.  
Also, we import `pprint` helper function from local `utils.py` file. We'll use `pprint` to display some structures.

In [ ]:
import os
import fhirpy

from utils import pprint

### 1.2 Create an instance of connection

To load data from FHIR server we should instantiate `FHIRClient` class from `fhirpy` package.  
Let's pass `url` and `authorization` arguments from environment.

In [ ]:
client = fhirpy.FHIRClient(
    url=os.environ['BOX_URL'] + '/fhir', 
    authorization=os.environ['BOX_AUTHORIZATION'])

Now, we are able to operate with FHIR resources using `client`.

## 2 Load data from the FHIR server

### 2.1 Load list of observations

[Observation](https://www.hl7.org/fhir/observation.html) is a central element in healthcare, used to support a diagnosis, monitor progress, determine baselines and patterns and even capture demographic characteristics. Most observations are simple name/value pair assertions with some metadata, but some observations group other observations together logically, or even are multi-component observations.

Usage of [Observation](https://www.hl7.org/fhir/observation.html) resource includes:

* Vital signs such as body weight, blood pressure, and temperature
* Laboratory Data like blood glucose, or an estimated GFR
* Imaging results like bone density or fetal measurements
* Devices Measurements such as EKG data or Pulse Oximetry data
* Clinical assessment tools such as APGAR or a Glasgow Coma Score
* Personal characteristics: such as eye-color
* Social histories like tobacco use, family support, or cognitive status
* Core characteristics like pregnancy status, or a death assertion


In [ ]:
observations = client.resources('Observation').limit(10).fetch()

This line fetched a portion of data in size of 10 from the server. You can see the result of its execution.  
It is a list containing resource representations for a `Observation`.

In [ ]:
observations

We can display all data from the `Observation`.

In [ ]:
pprint(observations[0])

## 3 Create Observation record

There are two patient's claims:  
* On Monday his temperature was 96.8 F and the blood pressure was 140/90.
* On Tuesday his temperature was 100.4 F and the blood pressure was 120/80.

The category of our claims is defined by the `category` field of the `Observation`. This field is of type [CodeableConcept](https://www.hl7.org/fhir/datatypes.html#CodeableConcept). 

[CodeableConcept](https://www.hl7.org/fhir/datatypes.html#CodeableConcept) consists of a terminology (`coding` field of type [Coding](https://www.hl7.org/fhir/datatypes.html#Coding)). We are going to use it in our formal description  
and a plain text problem explanation (`text` field of type [string](https://www.hl7.org/fhir/datatypes.html#string)). When none of the `coding` elements are marked as `userSelected`,  
the `text` (if present) is the preferred source of meaning.

Within [Coding](https://www.hl7.org/fhir/datatypes.html#Coding) we can refer to a particular `system` and `code`.

`Observation` defines its `category.coding` in terms of [observation-category](https://www.hl7.org/fhir/valueset-observation-category.html), so this is our `category.coding.system`.

[observation-category](https://www.hl7.org/fhir/valueset-observation-category.html) contains multiple concepts, you can read up on them in the official FHIR specification.  
Patient's claims contain information about the body temperature and the blood pressure, so we choose `vital-signs` as the `category.coding.code`.

From [vital-signs structure definition](https://www.hl7.org/fhir/vitalsigns.html) we can verify our `category` value (that we've figured out above) and find out what the `code` value is.

`Observation.category`:
* `coding.system` - `http://hl7.org/fhir/observation-category`
* `coding.code` - `vital-signs`

So we were right with our `category` configuration. Now let's find out the `code` value.

`Observation.code`:
* `coding.system` - `http://loinc.org`
* `coding.code` - one of the [allowed vital sign result types](https://www.hl7.org/fhir/valueset-observation-vitalsignresult.html), in our case it is `8310-5` for body temperature, `8480-6` for systolic blood pressure and `8462-4` for diastolic blood pressure.

Now we clarified `Observation` `category` and `code`. Let's create an `Observation` object.

Before we do that, please read a small note about `Observation` [status](https://www.hl7.org/fhir/valueset-observation-status.html) field: it is used to specify what the current status of an observation is.  
There are multiple values defined in the [FHIR specification]((https://www.hl7.org/fhir/valueset-observation-status.html)), some of them are:
* `registered` - The existence of the observation is registered, but there is no result yet available.
* `preliminary` - This is an initial or interim observation: data may be incomplete or unverified.
* `final` - The observation is complete.

In [ ]:
observation = client.resource(
    'Observation',
    status='preliminary',
    category=[{
        'coding': [{
            'system': 'http://hl7.org/fhir/observation-category',
            'code': 'vital-signs'
        }]
    }],
    code={
        'coding': [{
            'system': 'http://loinc.org',
            'code': '8310-5'
        }]
    })

`Observation` has a field called `valueQuantity` (of type [Quantity](https://www.hl7.org/fhir/datatypes.html#Quantity)) it contains current results of measurements performed.  
According to [vital-signs structure definition](https://www.hl7.org/fhir/vitalsigns.html) for `vital-signs` category this field can be populated with (remember, it is of [Quantity](https://www.hl7.org/fhir/datatypes.html#Quantity) type):
* `system` - `http://unitsofmeasure.org`
* `value` - represents one numeric value
* `code` - UCUM (stands for *The Unified Code for Units of Measure*) unit code from [Vital Signs Units](https://www.hl7.org/fhir/valueset-ucum-vitals-common.html)

So, on Monday (10-20-2018) our patient has a temperature of 96.8F let's write it in the observation

In [ ]:
observation['effectiveDateTime'] = '2018-10-20'

observation['valueQuantity'] = {
    'system': 'http://unitsofmeasure.org',
    'value': 96.8,
    'code': 'degF'
}

We've just created our first `Observation` object. Let's connect the `Observation` to the `Patient` from *Lab 1* before saving to the database.

In [ ]:
patient = client.resources('Patient').search(name=['John', 'Thompson']).first()
patient

`Patient` reference is stored in `Observation.subject`.

In [ ]:
observation['subject'] = patient.to_reference()

In [ ]:
observation.save()

Let's figure out what the current observation state is:

In [ ]:
pprint(observation)

**Exercise**: the patient weight is 85 kilograms, please, add this information as an observation using the temperature example above (remember using [vital-signs structure definition](https://www.hl7.org/fhir/vitalsigns.html), [allowed vital sign result types](https://www.hl7.org/fhir/valueset-observation-vitalsignresult.html) and [vital signs units](https://www.hl7.org/fhir/valueset-ucum-vitals-common.html)).

In [ ]:
weight_observation = client.resource(
    'Observation',
    status='preliminary',
    category=[{
        'coding': [
            # Write code here
        ]
    }],
    code={
        'coding': [
            # Write code here
        ]
    },
    effectiveDateTime='2018-10-20',
    valueQuantity={
        # Write code here
    },
    subject=patient.to_reference()
)

In [ ]:
pprint(weight_observation)

In [ ]:
weight_observation.save()

Let's create `Observation` object for the blood pressure.

Blood pressure value is built up of two parts: systolic and diastolic, and we have to specify both of them in the same `Observation`. For this purpose there is a special field in `Observation` object called `component`. Each `component` field consists of `code` and `valueQuantity`. For `Observation` with multiple components we have to specify section header code. For blood pressure this code is `55284-4` from `http://loinc.org` system.

In [ ]:
blood_pressure_observation = client.resource(
    'Observation',
    status='preliminary',
    category=[{
        'coding': [{
            'system': 'http://hl7.org/fhir/observation-category',
            'code': 'vital-signs'
        }]
    }],
    code={
        'coding': [{
            'system': 'http://loinc.org',
            'code': '55284-4'
        }]
    },
    component=[
        {
            'code': {
                'coding': [{
                    'system': 'http://loinc.org',
                    'code': '8480-6'
                }]
            },
            'valueQuantity': {
                'system': 'http://unitsofmeasure.org',
                'value': 140,
                'code': 'mmHg'
            }
        },
        {
            'code': {
                'coding': [{
                    'system': 'http://loinc.org',
                    'code': '8462-4'
                }]
            },
            'valueQuantity': {
                'system': 'http://unitsofmeasure.org',
                'value': 90,
                'code': 'mmHg'
            }
        }
    ],
    effectiveDateTime='2018-10-20',
    subject=patient.to_reference()
)

In [ ]:
blood_pressure_observation.save()

In [ ]:
pprint(blood_pressure_observation)

**Exercise**: Create observations for Tuesday values of the blood and the pressure using examples given above.


In [ ]:
tuesday_temperature_observation = client.resource(
    'Observation',
    # Write code here
)

In [ ]:
tuesday_blood_pressure_observaion = client.resource(
    'Observation',
    # Write code here
)

## 4 Work with related resources

We've just created our first two observations. Let's see how to make a search. 
We can use `patient` search parameter to find all the patient's observations.
There are multiple [search parameters](https://www.hl7.org/fhir/observation.html#search) defined for `Observation`. With `patient` we can search for observations which belong to the patient we've created before by giving patient's `id` as an argument. 

In [ ]:
client.resources('Observation').search(patient=patient['id']).fetch_all()

As you can see, there are multiple observations registered for the patient.

To do the reverse search for `Patient` based on a reference in `Observation` we can use [reverse chaining](https://www.hl7.org/fhir/search.html#has) with `_has` search parameter.  
`_has` allows to select resources based on the properties of resources which refer to them.

For example, query `GET /Patient?_has:Observation:patient:code=8310-5` will search for observations with `code=8310-5` (which is related to the body temperature). From every observation found retrieves a reference to the `Patient` this observation refers to. The patient is then included into the search result. Note that `code` here is not directly applies to `Observation` structure, it is defined as a [search parameter](https://www.hl7.org/fhir/observation.html#search) instead.

Let's build the same query with `fhir-py` API

In [ ]:
client.resources('Patient').has('Observation', 'patient', code='8310-5').fetch_all()

This query returns all of the temperature `Observation`'s we've created before.

### Summary

From this laboratory work you learned how to:
* load observations from the FHIR server 
* create Observation records
* search related resources
* display different resorce's attributes

<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>